In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import json
from pathlib import Path
import Packages.ClusteringHelper as ch

In [4]:
path_train = Path("D:\\Sgmon\\Documents\\Magistrale\\TESI\\ClusteringAndLinking\\aida-yago2-dataset\\encodings\\AIDA-YAGO2_train_encodings.jsonl")
path_testa = Path("D:\\Sgmon\\Documents\\Magistrale\\TESI\\ClusteringAndLinking\\aida-yago2-dataset\\encodings\\AIDA-YAGO2_testa_encodings.jsonl")
path_testb = Path("D:\\Sgmon\\Documents\\Magistrale\\TESI\\ClusteringAndLinking\\aida-yago2-dataset\\encodings\\AIDA-YAGO2_testb_encodings.jsonl")
raw_encodings_train = open(path_train, 'r').read()
raw_encodings_testa = open(path_testa, 'r').read()
raw_encodings_testb = open(path_testb, 'r').read()

In [5]:
jsonl_parsed_train = [json.loads(x) for x in raw_encodings_train.splitlines()]
jsonl_parsed_testa = [json.loads(x) for x in raw_encodings_testa.splitlines()]
jsonl_parsed_testb = [json.loads(x) for x in raw_encodings_testb.splitlines()]

In [6]:
jsonl_parsed = jsonl_parsed_train + jsonl_parsed_testa + jsonl_parsed_testb

In [7]:
embedding_titles = [x['title'].replace(" ", "_") for x in jsonl_parsed]

In [8]:
text, data = ch.read_aida_yago_conll(
    "D:\\Sgmon\\Documents\\Magistrale\\TESI\\ClusteringAndLinking\\aida-yago2-dataset\\AIDA-YAGO2-dataset.tsv")

In [9]:
ents_data = data[data['entities'] != '']
entities = ents_data['entities'].values

In [10]:
len(embedding_titles) == len(entities)

True

In [11]:
from sklearn.cluster import AgglomerativeClustering

clusterizator = AgglomerativeClustering(n_clusters=None, affinity='cosine', distance_threshold=0.035, linkage="single")

In [12]:
encodings = [x['encoding'] for x in jsonl_parsed]

In [13]:
clusters_list = clusterizator.fit_predict(encodings)

In [14]:
cluster_dict = {}
for i, cl in enumerate(clusters_list):
    try:
        cluster_dict[cl].append(entities[i])
    except:
        cluster_dict[cl] = [entities[i]]

In [15]:
from collections import Counter
for x in cluster_dict:
    cluster_dict[x] = Counter(cluster_dict[x])

In [16]:
import Packages.ClusteringHelper as ch
golden_standard_dict = ch.get_gold_standard_dict(data)

### B-CUBED

In [18]:
from tqdm import tqdm
# B-cubed - precision
bcubed_precision_num = 0
for gold_key in tqdm(golden_standard_dict.keys()):
    for lev_key in cluster_dict.keys():
        try:
            bcubed_precision_num = bcubed_precision_num + (pow(cluster_dict[lev_key][gold_key], 2) /
                                       sum([x for x in cluster_dict[lev_key].values()]))
        except:
            pass
bcubed_precision = bcubed_precision_num/sum([y for x in cluster_dict.values() for y in x.values()])
bcubed_precision

100%|██████████| 5598/5598 [00:30<00:00, 185.90it/s]


0.4299684051637073

In [19]:
# B-cubed - recall
bcubed_recall_num = 0
for gold_key in tqdm(golden_standard_dict.keys()):
    for lev_key in cluster_dict.keys():
        try:
            bcubed_recall_num = bcubed_recall_num + (pow(cluster_dict[lev_key][gold_key], 2) /
                                       golden_standard_dict[gold_key])
        except:
            pass
bcubed_recall = bcubed_recall_num/ents_data.shape[0]
bcubed_recall

100%|██████████| 5598/5598 [00:19<00:00, 289.74it/s]


0.9357471383729561

In [20]:
bcubed_f1 = (2 * (bcubed_recall * bcubed_precision)) / (
        bcubed_precision + bcubed_recall)
bcubed_f1

0.5892027906203907